In [22]:
%load_ext line_profiler

In [41]:
import numpy as np
import sklearn.metrics

def simulate(J, S=10, p=2, c=0.1, N_ed=2, p_ed=0.25, N_re=4, N_c=100, rng=(-5.12,5.12), d_attract=0.1, w_attract=0.2, h_repellant=0.1, w_repellant=10):

    theta = rng[0] + np.random.rand(S, p)*(rng[1] - rng[0])

    def J_cc(x, theta):
        """
        x: np.ndarray(m, p)
        """
        diff = sklearn.metrics.pairwise.euclidean_distances(theta, x, squared=True)
        return (-d_attract*np.exp(-w_attract*diff) + h_repellant*np.exp(-w_repellant*diff)).sum(axis=0)

    J_histories = np.zeros((N_ed, N_re, S, N_c))
    theta_histories = np.zeros((N_ed, N_re, S, N_c, p))
    #
    for l in range(N_ed):
        for k in range(N_re):
            for j in range(N_c):
                phi = np.random.uniform(low=-1, high=1, size=(S,p))
                phi /= np.linalg.norm(phi)
                theta = theta + c*phi
                J_theta = np.apply_along_axis(J, 1, theta) + J_cc(theta, theta)

                theta_new = theta + c*phi
                J_theta_new = np.apply_along_axis(J, 1, theta_new) + J_cc(theta_new, theta_new)

                to_run = np.where(J_theta_new < J_theta)[0]
                while len(to_run) > 0:
                    theta_old = theta_new.copy()
                    J_theta_old = J_theta_new.copy()
                    theta_new[to_run] = theta_old[to_run] + c*phi[to_run]
                    J_theta_new[to_run] = np.apply_along_axis(J, 1, theta_new[to_run]) + J_cc(theta_new[to_run], theta_new)
                    to_run = np.where(J_theta_new < J_theta_old)[0]

                theta = theta_new

                J_histories[l, k, :, j] = J_theta.copy()
                theta_histories[l, k, :, j, :] = theta.copy()

            I = np.argsort(J_histories[l, k].sum(axis=1)) # sort
            theta = np.concatenate((theta[I[:S//2]].copy(), theta[I[:S//2]].copy()))

        disperse = np.random.rand(S) < p_ed
        theta[disperse] = rng[0] + np.random.rand(disperse.sum(), p)*(rng[1] - rng[0])

    return J_histories, theta_histories

from losses import *
simulate(rastrigin)

(array([[[[38.3628017 , 38.11299722, 30.23408097, ..., 24.74930923,
           16.92063277, 17.10451547],
          [40.3736799 , 39.52449363, 38.72453499, ..., 26.25061091,
           27.91169969, 28.83694855],
          [43.21219565, 28.87445149, 27.28816344, ..., 26.93327103,
           26.77470492, 26.52373185],
          ...,
          [36.73288088, 22.9513527 , 23.90888396, ..., 20.08344183,
           18.03534444, 18.42606997],
          [34.84594925, 34.73442905, 30.70147589, ..., 31.68466411,
           34.37827772, 35.45168201],
          [60.43518084, 63.89838191, 52.6273451 , ..., 42.87700336,
           43.71196903, 44.23432988]],
 
         [[10.16851385, 13.84102742, 14.55792237, ...,  9.86084262,
           13.03935217, 16.71734692],
          [21.59946257, 21.41257623, 13.13907917, ..., 18.53300794,
           17.01140292, 12.9623705 ],
          [38.42703764, 26.59962138, 28.73007264, ...,  9.40828695,
            8.89837706,  8.94843035],
          ...,
          [35